In [ ]:
import pandas as pd
import os

players = ['주승우', '하영민', '헤이수스', '후라도']
files = ['_zone_avg.csv', '_zone_rate.csv']

index_list = ['타율 - ', '구사율 - ']
removing_index = ['너클볼', '우언', '주자없음', '주자있음', '득점권', '양타', '구종모름']
count_index = ['초구', '스트라이크 > 볼', '볼 > 스트라이크', '스트라이크 = 볼']
fast_ball_index = ['투심', '포심', '커터']
droping_columns = ['PA_' + str(i+1) for i in range(25)]

for player in players:
    for i in range(2):
        # 파일 읽기
        df = pd.read_csv('../pitcher_data/' + player + files[i])
        df = df.drop(columns=droping_columns, errors='ignore')

        df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: x.replace(index_list[i], '') if index_list[i] in x else x)

        # 제거할 행 필터링 (첫 번째 열의 값이 `removing_index`에 포함된 경우)
        removing_rows = [index for index in removing_index]
        df = df[~df.iloc[:, 0].isin(removing_rows)]

        # count 행을 합쳐서 하나의 행으로 만들기
        count_rows = [index for index in count_index]
        if all(row in df.iloc[:, 0].values for row in count_rows):
            count_sum = df[df.iloc[:, 0].isin(count_rows)].iloc[:, 1:].mean()
            count_sum = count_sum.round(3)
            count_sum_row = pd.DataFrame([['카운트'] + count_sum.tolist()], columns=df.columns)
            df = pd.concat([df, count_sum_row], ignore_index=True)
            df = df[~df.iloc[:, 0].isin(count_rows)]

        existing_fast_ball = [row for row in fast_ball_index if row in df.iloc[:, 0].values]
        if existing_fast_ball:  # 존재하는 값이 하나라도 있다면
            # 기존 값으로 작업 진행
            fastball_sum = df[df.iloc[:, 0].isin(existing_fast_ball)].iloc[:, 1:].mean()
            fastball_sum = fastball_sum.round(3)
            fastball_sum_row = pd.DataFrame([['패스트볼'] + fastball_sum.tolist()], columns=df.columns)
            
            # 기존 데이터프레임에 추가
            df = pd.concat([df, fastball_sum_row], ignore_index=True)
            df = df[~df.iloc[:, 0].isin(existing_fast_ball)]  # 기존 행 삭제

        df.loc[df.iloc[:, 0] == '2S 이후', df.columns[0]] = '결정구'
        df.to_csv(player + files[i], index=False)
